In [7]:
from CausalBenchmark.data_generation_framework import *
from CausalBenchmark.parameters import load_parameters
import numpy as np

In [11]:
load_parameters()

In [12]:
simulated_data = created_simulated_data_from_random_covariates()
simulated_data

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X16,X17,X18,X19,NOISE,T,Y0,Y1,Y,treatment_effect
418,-6.403158,4.592048,-1.522418,-1.299657,-6.957772,-3.994356,9.919837,6.283871,9.015064,2.835756,...,-2.928073,1.762605,0.105414,-0.154833,-1.472,0,-1.459304,2.428907,-1.459304,3.888211
849,5.322637,4.225285,5.769789,3.801157,-5.761317,-1.652710,4.836235,-2.993497,-0.674343,4.087157,...,8.259191,-1.173368,-2.597939,7.807727,-0.167,0,-0.389048,3.796659,-0.389048,4.185707
567,-3.673565,0.150636,-3.165100,-3.363926,-0.850881,-4.496947,2.282083,-3.435106,0.304759,4.909422,...,2.291035,7.147904,-9.566732,6.154265,0.697,1,0.716918,6.323565,6.323565,5.606646
780,-0.800908,-5.177946,2.533547,-7.579248,2.496652,1.834097,-1.523576,3.205750,5.959814,1.143024,...,-4.142368,1.803815,-4.244360,6.374707,0.542,0,0.295060,3.929255,0.295060,3.634195
112,1.541685,0.406210,6.861003,3.358003,5.753057,-5.574996,9.768892,4.952890,2.975175,-5.630756,...,-13.584323,1.775642,10.676105,3.110839,0.561,1,-0.584135,2.212605,2.212605,2.796740
168,3.261883,-2.195885,-7.395684,-9.501900,4.713136,-2.514541,5.912981,0.302223,-4.877345,-2.489233,...,3.942497,-4.918090,-3.977621,-3.534925,0.080,1,0.088633,-0.452014,-0.452014,-0.540647
961,-4.957143,1.877188,4.292785,-0.473219,0.418813,4.302545,-0.401507,3.592075,-3.241780,-4.448789,...,-7.747934,7.697852,2.301758,-7.824940,1.098,1,1.474271,5.579756,5.579756,4.105485
236,5.602243,-1.473969,-6.812079,1.041850,1.252158,3.427725,-4.143737,-6.492963,8.582138,6.948246,...,2.404465,5.261421,-2.465782,1.958515,0.323,1,0.326081,4.771798,4.771798,4.445717
51,4.774410,-2.592611,-3.177588,-10.012024,-1.382511,-0.273681,-5.543943,-3.863462,-11.275355,3.763346,...,1.407048,4.951525,-2.728951,10.121143,1.940,1,2.868329,13.984698,13.984698,11.116370
389,1.498655,5.951163,3.954562,0.448552,0.688441,-4.139355,0.133319,4.411500,1.391318,-3.560551,...,-4.086540,-2.713593,12.777674,-7.680412,0.140,1,0.194035,4.824455,4.824455,4.630420
